# 科学技術計算 課題２


In [53]:
import math
import cmath
from typing import Tuple, List
import numpy as np
import scipy

import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams["savefig.bbox"] = "tight"

rng = np.random.default_rng()

from typing import Tuple
import sys
import math
import scipy

import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams["savefig.bbox"] = "tight"
plt.gray()


<Figure size 640x480 with 0 Axes>

## 課題02-2

In [ ]:
a = 18.0
b = 2.0**57
c = (a + b) - b
print(c)

def float64_to_binary(f: float | np.float64,separate: bool = False) -> str | Tuple[str, str, str]:
    """convert float64 to 64 bits as string

        Args:
        f (float | np.float64): IEEE754 binary64 floating point number
        separate (bool, optional): return tuple of (sign bit, exponent bits, mantissa bits) if True,
            return 64 bits if False. Defaults to False.

    Returns:
        str or Tuple of str: bit string of f
    """
    assert isinstance(f, float)
    from struct import pack, unpack

    # see https://note.nkmk.me/python-float-hex/
    s = format(unpack('>Q', pack('>d', f))[0], "064b")

    if separate:
        return s[0:1], s[1:12], s[12:]
    else:
        return s

def sign_str_to_str(s: str) -> str:
    """sign bit to +/-

    Args:
        s (str): "0" or "1"

    Returns:
        str: "+" if s is "0" else "-" (if s is "1")
    """
    assert isinstance(s, str)
    assert len(s) == 1
    assert s in ["0", "1"]

    return "+" if s == "0" else "-"

def exponent_str_to_int(
        e: str,
        len_e: int = 11,
        bias: int = 1023
    ) -> Tuple[int, bool]:
    """exponent string to exponent integer

    Args:
        e (str): exponent string consiting of of "0" and "1"
        len_e (int, optional): the number of bits for the exponent. Defaults to 11.
        bias (int, optional): bias to exponent. Defaults to 1023.

    Returns:
        Tuple[int, bool]: exponent in decimal integer, and True if normalized number else False
    """
    assert isinstance(e, str)
    assert len(e) == len_e
    for bit in e:
        assert bit in ["0", "1"]

    p = sum([2**i * int(bit) for i, bit in enumerate(e[::-1])])
    if p > 0:
        p -= bias
        is_normalized = True
    else:
        p = -bias + 1
        is_normalized = False
    if p > bias:
        p = np.nan
    return p, is_normalized


def mantissa_str_to_float(
        m: str,
        is_normalized: bool,
        len_m: int = 52
    ) -> float:
    """mantissa string to mantissa integer

    Args:
        m (str): mantissa string consiting of of "0" and "1"
        is_normalized (bool): True if m is normalized number else False
        len_m (int, optional): the number of bits for the mantissa. Defaults to 52.

    Returns:
        float: mantissa in decimal float
    """
    assert isinstance(m, str)
    assert len(m) == len_m
    for bit in m:
        assert bit in ["0", "1"]

    p = 1 if is_normalized else 0
    p += sum([2 ** -(i + 1) * int(bit) for i, bit in enumerate(m)])

    return p

def sem_to_str(s: str, e: str, m: str, n_spaces: int = 0) -> str:
    """convert output strings of `float64_to_binary()` with `separate=True` to a human-readable string

        Args:
        s, e, m (str): bit strings of sign, exponent, and mantissa.
        n_spaces (int, optional): the number of leading white spaces on the left. Defaults to 0.

    Returns:
        str: human-readable string represetnation of a number represented by s, e, and m
    """
    s = sign_str_to_str(s)
    e, is_normalized = exponent_str_to_int(e)
    m = mantissa_str_to_float(m, is_normalized)

    n_spaces = " " * n_spaces

    if e is not np.nan:
        return f"{n_spaces}{s} 2^{e:<+7d} * {m}"
    else:
        return f"{n_spaces}{s} nan/inf   * {m}"

print(f"decimal  s exponent    mantissa")
for f in [
        a,
        b,
        a + b,
        -b,
        c,
        
    ]:
    s, e, m = float64_to_binary(f, separate=True)
    print(f"{f:.2f} --> {s} {e} {m}")
    print(sem_to_str(s, e, m, n_spaces=9))

print("aaaaaaaaaaaaa")
#a, bをIEEE754 binary64のビット表現に変換、符号・指数・仮数を確認
print(float64_to_binary(a))
print("a =",float64_to_binary(a, separate=True))

print(float64_to_binary(b))
print("b =",float64_to_binary(b, separate=True))



32.0
decimal  s exponent    mantissa
18.00 --> 0 10000000011 0010000000000000000000000000000000000000000000000000
         + 2^+4      * 1.125
144115188075855872.00 --> 0 10000111000 0000000000000000000000000000000000000000000000000000
         + 2^+57     * 1.0
144115188075855904.00 --> 0 10000111000 0000000000000000000000000000000000000000000000000001
         + 2^+57     * 1.0000000000000002
-144115188075855872.00 --> 1 10000111000 0000000000000000000000000000000000000000000000000000
         - 2^+57     * 1.0
32.00 --> 0 10000000100 0000000000000000000000000000000000000000000000000000
         + 2^+5      * 1.0
aaaaaaaaaaaaa
0100000000110010000000000000000000000000000000000000000000000000
a = ('0', '10000000011', '0010000000000000000000000000000000000000000000000000')
0100001110000000000000000000000000000000000000000000000000000000
b = ('0', '10000111000', '0000000000000000000000000000000000000000000000000000')


: 

## 課題02-5

In [ ]:
#愚直に実装した数値的に不安定なsoftmax
def unstable_softmax(x: np.ndarray,) -> np.ndarray:
    a = np.sum(np.exp(x))  
    y = np.exp(x) / a  

    return y

def stable_softmax(x: np.ndarray,) -> np.ndarray:
    max = np.max(x)
    exp_x = np.exp(x - max)
    sum_exp_x = np.sum(exp_x)

    y = np.exp(x - max - np.log(sum_exp_x)) #log


    return y

rng = np.random.default_rng()
a = rng.integers(800,1000 ,size=10)

y = unstable_softmax(a)
print(y)
for i in y:
    print(f"{i:.15e}")

print("nyaaaaaaaaa")

z = stable_softmax(a)
for i in z:
    print(f"{i:.15e}")

print(np.sum(y))  # 1.0 になる！
print(np.sum(z))

[nan nan nan nan nan nan nan nan nan nan]
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nyaaaaaaaaa
1.910846313178810e-64
1.636305014825742e-04
9.654347741141197e-33
1.492117033166009e-07
2.428491965056770e-02
7.623358762734857e-19
4.877756503181233e-01
1.989909381893469e-41
4.877756503181233e-01
4.932492211737243e-44
nan
1.0000000000000002


C:\Users\sayuk\AppData\Local\Temp\ipykernel_28356\4050564820.py:3: RuntimeWarning: overflow encountered in exp
  a = np.sum(np.exp(x))
C:\Users\sayuk\AppData\Local\Temp\ipykernel_28356\4050564820.py:4: RuntimeWarning: overflow encountered in exp
  y = np.exp(x) / a
C:\Users\sayuk\AppData\Local\Temp\ipykernel_28356\4050564820.py:4: RuntimeWarning: invalid value encountered in divide
  y = np.exp(x) / a


: 

## 課題02-6

In [ ]:
def machine_epsilon(dtype):
    eps = dtype(1)  # ε ← 1
    one = dtype(1)

    while one + eps > one:
        print(f"{eps:.15e}")  # ← e表記で表示
        eps = eps / dtype(2)  # εを半分にする

    eps = eps * dtype(2)  # 最後に2倍して戻す
    print("Machine epsilon =", f"{eps:.15e}")
    
    return eps

# それぞれの型で実行
print("np.float64")
machine_epsilon(np.float64)

print("np.float32")
machine_epsilon(np.float32)

print("np.float16")
machine_epsilon(np.float16)

print(np.finfo(np.float64))
print(np.finfo(np.float32))
print(np.finfo(np.float16))


np.float64
1.000000000000000e+00
5.000000000000000e-01
2.500000000000000e-01
1.250000000000000e-01
6.250000000000000e-02
3.125000000000000e-02
1.562500000000000e-02
7.812500000000000e-03
3.906250000000000e-03
1.953125000000000e-03
9.765625000000000e-04
4.882812500000000e-04
2.441406250000000e-04
1.220703125000000e-04
6.103515625000000e-05
3.051757812500000e-05
1.525878906250000e-05
7.629394531250000e-06
3.814697265625000e-06
1.907348632812500e-06
9.536743164062500e-07
4.768371582031250e-07
2.384185791015625e-07
1.192092895507812e-07
5.960464477539062e-08
2.980232238769531e-08
1.490116119384766e-08
7.450580596923828e-09
3.725290298461914e-09
1.862645149230957e-09
9.313225746154785e-10
4.656612873077393e-10
2.328306436538696e-10
1.164153218269348e-10
5.820766091346741e-11
2.910383045673370e-11
1.455191522836685e-11
7.275957614183426e-12
3.637978807091713e-12
1.818989403545856e-12
9.094947017729282e-13
4.547473508864641e-13
2.273736754432321e-13
1.136868377216160e-13
5.684341886080801e-14

: 